<a href="https://colab.research.google.com/github/higherbar-ai/ai-workflows/blob/main/src/example-doc-extraction-templated.ipynb" target="_parent"><img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg"/></a>

# About this notebook

This notebook provides an example of how the `ai-workflows` package can be used to extract arbitrary structured data from an arbitrary number of unstructured documents. It prompts you to upload an `.xlsx` file with a data extraction template, then one or more files from which to extract data. The template follows a specific format, as in [the example template here](https://docs.google.com/spreadsheets/d/1jI0hXKS_vPlbaFH-wwYTVBVYShjDS4m3N14fU21JfK4/edit) (which you can copy and edit to adapt to your needs).

## Configuration

This notebook requires different settings depending on which AI service providers you want to use. If you're running in Google Colab, you configure these settings as "secrets"; just click the key icon in the left sidebar (and, once you create a secret, be sure to click the toggle to give the notebook access to the secret). If you're running this notebook in a different environment, you can set these settings in a `.env` file; the first time you run, it will write out a template `.env` file for you to fill in and direct you to its location. 

Following are the settings, regardless of the environment.

### OpenAI (direct)

To use OpenAI directly:

* `openai_api_key` - your OpenAI API key (get one from [the OpenAI API key page](https://platform.openai.com/api-keys), and be sure to fund your platform account with at least $5 to allow GPT-4o model access)
* `openai_model` (optional) - the model to use (defaults to `gpt-4o`)

### OpenAI (via Microsoft Azure)

To use OpenAI via Microsoft Azure:

* `azure_api_key` - your Azure API key
* `azure_api_base` - the base URL for the Azure API
* `azure_api_engine` - the engine to use (a.k.a. the "deployment")
* `azure_api_version` - the API version to use

### Anthropic (direct)

To use Anthropic directly:

* `anthropic_api_key` - your Anthropic API key
* `anthropic_model` - the model to use

### LangSmith (for tracing)

Optionally, you can add [LangSmith tracing](https://langchain.com/langsmith):

* `langsmith_api_key` - your LangSmith API key

## Setting up the runtime environment

This next code block installs all necessary Python and system packages into the current environment.

**If you're running in Google Colab and it prompts you to restart the notebook in the middle of the installation steps, just click CANCEL.**

In [ ]:
# install Google Colab Support and ai_workflows package
%pip install colab-or-not py-ai-workflows[docs]

# download NLTK data
import nltk
nltk.download('punkt', force=True)

# set up our notebook environment (including LibreOffice)
from colab_or_not import NotebookBridge
notebook_env = NotebookBridge(
    system_packages=["libreoffice"],
    config_path="~/.hbai/ai-workflows.env",
    config_template={
        "openai_api_key": "",
        "openai_model": "",
        "azure_api_key": "",
        "azure_api_base": "",
        "azure_api_engine": "",
        "azure_api_version": "",
        "anthropic_api_key": "",
        "anthropic_model": "",
        "langsmith_api_key": "",
    }
)
notebook_env.setup_environment()

## Initializing for AI workflows

The next code block initializes the notebook by loading settings and initializing the LLM interface.

In [ ]:
from ai_workflows.llm_utilities import LLMInterface
from ai_workflows.document_utilities import DocumentInterface

# read all supported settings
openai_api_key = notebook_env.get_setting('openai_api_key')
openai_model = notebook_env.get_setting('openai_model', 'gpt-4o')
azure_api_key = notebook_env.get_setting('azure_api_key')
azure_api_base = notebook_env.get_setting('azure_api_base')
azure_api_engine = notebook_env.get_setting('azure_api_engine')
azure_api_version = notebook_env.get_setting('azure_api_version')
anthropic_api_key = notebook_env.get_setting("anthropic_api_key")
anthropic_model = notebook_env.get_setting("anthropic_model")
langsmith_api_key = notebook_env.get_setting('langsmith_api_key')

# complain if we don't have the bare minimum to run
if (not openai_api_key
        and not (azure_api_key and azure_api_base and azure_api_engine and azure_api_version)
        and not (anthropic_api_key and anthropic_model)):
    raise Exception('We need settings set for OpenAI access (direct or via Azure) or for Anthropic access (direct). See the instructions above for more details.')

# initialize LLM interface
llm = LLMInterface(openai_api_key=openai_api_key, openai_model=openai_model, azure_api_key=azure_api_key, azure_api_base=azure_api_base, azure_api_engine=azure_api_engine, azure_api_version=azure_api_version, temperature = 0.0, total_response_timeout_seconds=600, number_of_retries=2, seconds_between_retries=5, langsmith_api_key=langsmith_api_key, anthropic_api_key=anthropic_api_key, anthropic_model=anthropic_model)

# initialize our document processor
doc_interface = DocumentInterface(llm_interface=llm)

# report success
print("Initialization successful.")

## Prompting for input files

This next code block prompts you to upload or select your input files:

1. An `.xlsx` file with the document extraction template. This template specifies information about the document(s) you want to process and the information you wish to extract from those documents; it follows a very specific format, as in [the example template here](https://docs.google.com/spreadsheets/d/1jI0hXKS_vPlbaFH-wwYTVBVYShjDS4m3N14fU21JfK4/edit). The easiest thing to do is (a) open [the example template](https://docs.google.com/spreadsheets/d/1jI0hXKS_vPlbaFH-wwYTVBVYShjDS4m3N14fU21JfK4/edit), (b) click on "File" > "Make a copy" to make your own copy, and (c) edit the template to meet your needs, (d) click on "File" > "Download" > "Microsoft Excel (.xlsx)" to download the template to your computer, and (e) upload or select the template here.

2. The document(s) you want to process. If you only want to process a single document, just upload or select that. If you want to process multiple, upload or select them all or compress them together into a single `.zip` file and upload or select that.

In [ ]:
template_file = ""
while True:
    # prompt for a .xlsx file
    selected_files = notebook_env.get_input_files(".xlsx file with processing template:")

    # complain if we didn't get just a single .xlsx file
    if len(selected_files) != 1 or not selected_files[0].lower().endswith('.xlsx'):
        print()
        print('Please upload a single .xlsx file with your processing template to continue.')
        print()
    else:
        # fetch the path of the uploaded file
        template_file = selected_files[0]

        # break from loop
        break

# report out on the processing template
print(f'Will process using template: {template_file}')

# prompt for one or more files to process
files_to_process = notebook_env.get_input_files("Document(s) to process (.zip file for multiple):")

# report out on the files we plan to process
for file_to_process in files_to_process:
    if file_to_process.lower().endswith('.zip'):
        print(f'Will process all files within: {file_to_process}')
    else:
        print(f'Will process: {file_to_process}')

## Extracting data

This next code block starts by reading the supplied template to guide the data extraction process. It then processes each file one-by-one, unzipping `.zip` files into a temporary directory as needed, and aggregates all the results into a single list of rows.

In [ ]:
import tempfile
import zipfile
import os
from openpyxl import load_workbook

# load the template from the first worksheet in the uploaded workbook
wb = load_workbook(template_file)
sheet = wb.worksheets[0]

# extract the template from the first two columns of the worksheet
template_dict = {}
for row in sheet.iter_rows(values_only=True):
    # extract the first two columns
    key_cell = row[0] if len(row) > 0 else None
    val_cell = row[1] if len(row) > 1 else None

    # only proceed if both key and value cells have content — and if it's not a duplicate key
    if key_cell and val_cell:
        key = str(key_cell).strip()
        value = str(val_cell).strip()
        if key not in template_dict:
            template_dict[key] = value

# organize template data for processing
if 'meta_document_desc' not in template_dict:
    raise Exception('Processing template must include a "meta_document_desc" row that describes the documents being processed.')
document_desc = template_dict['meta_document_desc']
if 'meta_row_desc' not in template_dict:
    raise Exception('Processing template must include a "meta_row_desc" row that describes the rows of data you want extracted.')
row_desc = template_dict['meta_row_desc']
page_by_page = False
if 'meta_opt_page_by_page' in template_dict:
    page_by_page = (template_dict['meta_opt_page_by_page'] == '1' or template_dict['meta_opt_page_by_page'].lower() == 'true')
output_data = {}
for k, v in template_dict.items():
    if not k.startswith("meta_"):
        output_data[k] = v

# next, process files, with zip files unzipped into a temporary directory
all_rows = []
with tempfile.TemporaryDirectory() as temp_dir:
    # tally up all files, unzipping as needed
    all_files = []
    for file_to_process in files_to_process:
        if file_to_process.lower().endswith('.zip'):
            # if it's a .zip file, unzip it into the temporary directory
            print(f'Unzipping {file_to_process}')
            with zipfile.ZipFile(file_to_process, 'r') as zip_ref:
                zip_ref.extractall(temp_dir)
        else:
            # just add the file to the list of files to process
            all_files.append(file_to_process)
    # add all unzipped files to the list of files to process (ignoring hidden files)
    for root, dirs, files in os.walk(temp_dir):
        for unzipped_file in files:
            unzipped_file_path = os.path.join(root, unzipped_file)
            if not unzipped_file.startswith('.'):
                all_files.append(unzipped_file_path)

    # process each file
    for file_to_process in all_files:
        filename = os.path.basename(file_to_process)
        print(f'Processing {filename}...')

        json_context = f"""The file contains the following: {document_desc}"""

        json_job = f"""Your job is to extract a series of row objects from the file's content, and to return them all in a specific JSON format. Each row object should represent the following: {row_desc}"""

        json_output_spec = f"""Return JSON with the following fields (and only the following fields):\n\n* `rows` (list): The list of rows extracted, or an empty list if none found. Each row should contain the following fields:"""
        for k, v in output_data.items():
            json_output_spec += f"""\n\n  * `{k}` (string): {v}"""

        # process the file
        all_responses = doc_interface.convert_to_json(file_to_process, json_context, json_job, json_output_spec, markdown_first=not page_by_page)

        # combine all responses into a single list of rows
        merged_responses = doc_interface.merge_dicts(all_responses)
        rows = merged_responses['rows']

        # output and save results
        print(f"  Extracted {len(rows)} row{'s' if len(rows) != 1 else ''}")
        all_rows.append((filename, rows))

## Outputting extracted data

This final code block outputs the extracted data to `extracted_data.csv`, with the filename in column A and the requested output columns thereafter.

If you're running in Google Colab, this `.csv` file will be saved into the content folder. Find, view, or download it by clicking on the folder icon in the left sidebar.

If you're running elsewhere, it will be saved into an `ai-workflows` subdirectory created off of your user home directory.

In [ ]:
import csv

# output files to ~/ai-workflows directory if local, otherwise /content if Google Colab
output_path_prefix = notebook_env.get_output_dir(not_colab_dir="~/ai-workflows", colab_subdir="")

# output .csv file with extracted data, with filename in column A and the output columns thereafter
output_csv_path = os.path.join(output_path_prefix, 'extracted_data.csv')
with open(output_csv_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['filename'] + list(output_data.keys()))
    for filename, rows in all_rows:
        for row in rows:
            csvwriter.writerow([filename] + [row.get(k, '') for k in output_data.keys()])

# report out on the output file
print(f'Extracted data saved to: {output_csv_path}')